# Neural network based classifier
前処理
1. 16ごとに切り出し、それを一つのカテゴリとしてラベルエンコーディング
2. fftをかける
3. PCAを100次元でかける

学習器
- MLPによる学習。Activationには2次元でApproximateしたSwishを使う


## 結果
精度99.9%  
なんかリークしてる？  
それとも機械学習を使うべきでない決定的な問題なのか？

In [20]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, MaxPooling1D, GlobalAveragePooling1D, AveragePooling1D, Conv2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [21]:
MAX_SEQ = 30000
K = 16
N = 16000

In [5]:
strain2num = {'B.1.1.7': 0, 'B.1.427': 1, 'B.1.526': 2, 'P.1': 3}

In [6]:
with open("dataset/Challenge.fa", "r") as f:
    a = f.readlines()
    labels = []
    eds = []

    for i in range(int(N/2)):
        labels.append(strain2num[a[i*2][1:].split("_")[0]])

    unique_seq = set()
    sequences = []
    for i in range(int(N/2)):
        seq = list(map(lambda x : "".join(x), 
                       np.append(np.array(list(a[i*2+1])), 
                                ["O" for i in range(MAX_SEQ - len(a[i*2+1]))])
                       .reshape(8, -1)
                       .transpose()
                       .tolist()))
        sequences.append(seq)
        unique_seq |= set(seq)

ラベルエンコーディングする

In [7]:
enc = LabelEncoder()
enc.fit(list(unique_seq))
ids = enc.transform(np.array(sequences).reshape(-1)).reshape(len(sequences), -1)

XにはFFTをかける。  
YにはOneHotEncoding

In [8]:
f = np.abs(np.fft.fft(ids)) ** 2
X = f[:, 1:int(f.shape[1]/2)]

enc = OneHotEncoder()

Y = np.array(labels).reshape(-1,1)

Y = enc.fit_transform(Y)
Y = np.array(Y.toarray(), dtype="int32")

Testとtrainに分ける

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0, shuffle=True)

特徴料抽出をPCAで行う。

In [10]:
pca = PCA(n_components=100)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

標準化

In [11]:
mean, std = x_train.mean(), x_train.std()
standardize = lambda x: (x - mean) / std

x_train = standardize(x_train)
x_test = standardize(x_test)

In [14]:
import cf

In [92]:
scale = pow(2.0, 40)
mc = 4
service = cf.Py_CKKSEncryptionService(scale=scale, multiplication_capability=mc)

In [93]:
enc_x = service.encrypt(x_train[0].tolist())

Approximated swishの実装

In [22]:
# Refarences:
# Highly Accurate CNN Inference Using Approximate Activation Functions over Homomorphic Encryption
# Takumi Ishiyama, Takuya Suzuki, Hayato Yamana, Waseda Univ.
# https://arxiv.org/pdf/2009.03727.pdf


class ApproximatedSwish(tf.keras.layers.Layer):
    def __init__(self, degree=2, ranges=4, **kwargs):
        super(ApproximatedSwish, self).__init__(**kwargs)
        if degree == 2:
            if ranges == 4:
                self._func = lambda x: 0.12592 + 0.5*x + 0.145276*(x**2)
            elif ranges == 6:
                self._func = lambda x: 0.0851505 + 0.5*x + 0.344125*(x**2)
            else:
                raise ValueError(f"Approximated swish range is 4 or 6. not {ranges}")
        elif degree == 4:
            if ranges == 4:
                self._func = lambda x: 0.03347 + 0.5*x + 0.19566*(x**2) - 0.005075*(x**4)
            elif ranges == 6:
                self._func = lambda x: 0.1198 + 0.5*x + 0.1473*(x**2) - 0.002012*(x**4)
            else:
                raise ValueError(f"Approximated swish range is 4 or 6. not {ranges}")

    def call(self, x):
        return self._func(x)

MLPの構築

In [23]:
dropout_rate = 0.2

model = Sequential([
#     Dense(128, input_shape=(x_train.shape[1],)),
#     ApproximatedSwish(degree=2, ranges=4),
    Dense(4),
    Activation('softmax'),
])

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(x_train, np.argmax(y_train, axis=-1))

/home/coder/venv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [42]:
bias = clf.intercept_[:, None].tolist()
coef = clf.coef_.tolist()

In [95]:
enc_coef = service.encrypt_2d(coef)

In [96]:
enc_bias = service.encrypt_2d(bias)

In [101]:
service.decrypt(enc_bias[0].rotate_right(1))

[3.3380290371981435e-07,
 0.052437836088829966,
 -1.3373723099631063e-07,
 1.3371937874695886e-08,
 -5.3890083869946715e-09,
 7.023544279318937e-09,
 2.0929212971074216e-09,
 8.489326698454268e-09,
 9.968826040894275e-10,
 -1.2396024885764894e-08,
 4.507591160561059e-09,
 5.09872437912286e-10,
 -1.727730912490955e-09,
 -3.1449060556988807e-09,
 -1.0340864799136926e-08,
 -5.701988258374433e-09,
 4.738485524242333e-09,
 3.870108958053136e-09,
 -1.3869126843850524e-09,
 -2.0698413938550514e-09,
 -4.255990353265982e-08,
 -6.468448910716528e-09,
 7.852354331511411e-09,
 7.370296337700478e-10,
 -6.006946768477606e-09,
 3.139942917073391e-09,
 -4.36037008948956e-08,
 7.187031585274663e-09,
 -1.2994861104474126e-09,
 -7.236395000816766e-09,
 -5.9165796461399966e-09,
 1.1447294543307765e-08,
 -9.941359769620385e-10,
 8.781303855209055e-10,
 1.2818794535564628e-08,
 -1.8553419528213748e-08,
 2.321840342482711e-08,
 1.4749721863728928e-08,
 3.047920987140645e-09,
 -2.9624630133446284e-09,
 -5.672

In [86]:
len(service.decrypt(mlt))

4096

In [109]:
import copy

In [196]:
math.log2(100)

6.643856189774724

In [201]:
result = []
for i in range(4):
    mlt = enc_x * enc_coef[i]
    ones = [1 for _ in range(100)]
    ones.extend([0 for _ in range(8192 - 100)])
    out = mlt * ones
    for j in range(int(math.log2(100) + 2)):
        out += out.rotate_right(-2**j)
    out += enc_bias[i]
    result.append(out)

In [179]:
np.sum(service.decrypt(mlt)[:10]), np.sum((coef[3] * x_train[0])[:10])

(6.228800568940001, 6.22880056560072)

In [209]:
for i in range(4):
    print(service.decrypt(result[i])[0])

-2.1326817286751183
-3.3647174334594343
-1.5689258783953184
7.066307888066209


In [166]:
np.abs(service.decrypt(mlt)[:100] - (coef[i] * x_train[0]))

array([2.28391872e-09, 1.85631738e-09, 1.73106418e-09, 5.65095748e-10,
       1.18164800e-09, 1.26875593e-10, 1.75656346e-09, 3.49016687e-10,
       2.71218292e-09, 3.21723181e-09, 4.24602775e-09, 5.75671663e-10,
       1.29708913e-09, 1.07087283e-09, 1.72685025e-09, 6.07111571e-09,
       3.30946460e-09, 4.01418898e-11, 1.84591419e-09, 5.25223847e-10,
       8.21243245e-09, 1.75893195e-10, 1.38256316e-10, 8.72993010e-11,
       9.12373286e-10, 4.97890990e-09, 4.30342132e-09, 4.09291151e-09,
       1.32150776e-09, 1.16172313e-09, 3.09126688e-09, 2.77666643e-09,
       7.55130317e-09, 9.10779432e-10, 9.22226321e-11, 5.21538663e-10,
       2.44969425e-09, 3.27367052e-09, 1.96285911e-10, 1.14557735e-09,
       3.70766372e-11, 8.54077982e-10, 3.46191147e-09, 1.48231244e-09,
       3.84159741e-10, 3.20423321e-09, 4.72748365e-09, 1.64364655e-09,
       2.90758419e-10, 6.96649647e-10, 4.99270439e-10, 7.97998857e-10,
       3.71174539e-09, 2.11327980e-10, 1.35838306e-09, 9.18294212e-11,
      

import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [122]:
y_train[0]

array([0, 0, 0, 1], dtype=int32)

In [123]:
clf.predict_proba(x_train[0,None])

array([[1.01114651e-04, 2.94950797e-05, 1.77684398e-04, 9.99691706e-01]])

In [69]:
pred = clf.predict(x_test)
accuracy_score(pred, np.argmax(y_test, axis=-1))

ValueError: Expected 2D array, got 1D array instead:
array=[-1.66605750e+00 -1.72525403e+00 -3.90629182e+00  3.32204777e+00
 -2.79188449e-01  3.66350560e-01 -1.73597236e+00  2.15884794e-01
 -1.30318097e+00  1.91575669e+00 -4.40414942e-01 -2.07342032e-01
  3.46309608e-01 -1.26102551e-01  4.24992040e-01  7.64531048e-02
 -9.82207434e-02  1.96906592e-01 -1.74955844e-01 -9.77044046e-02
  1.42628490e-01 -2.22131112e-03  3.22919995e-02  9.78402175e-02
  8.81032676e-02  1.50060653e-01  1.66693730e-02  4.40384487e-01
  5.90261352e-01 -6.74877227e-01 -7.22888815e-01 -9.56445215e-02
 -1.47500979e-01 -3.22529262e-02 -1.20537670e-01  9.65661917e-03
 -1.51723788e-01  2.90838795e-01 -2.70930306e-01  4.15923214e-03
 -7.61022328e-02 -5.97852449e-02 -1.28747860e-01 -1.69575266e-01
 -7.78130220e-02  2.79152004e-02  8.83382561e-02 -6.15603695e-02
 -9.77927333e-02 -3.00943415e-02  7.12580593e-03  1.06908904e-01
 -1.02943515e-01 -4.68999377e-02 -1.22908151e-01  4.45831184e-02
 -2.97042340e-03  1.87758908e-0

訓練スタート

In [20]:
history = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=50,
#     validation_data=(x_test, y_test),
)

Epoch 1/50
225/225 [==============================] - 1s 788us/step - loss: 1.3762 - accuracy: 0.3988
Epoch 2/50
225/225 [==============================] - 0s 781us/step - loss: 0.6496 - accuracy: 0.9420
Epoch 3/50
225/225 [==============================] - 0s 759us/step - loss: 0.3626 - accuracy: 0.9886
Epoch 4/50
225/225 [==============================] - 0s 821us/step - loss: 0.2345 - accuracy: 0.9967
Epoch 5/50
225/225 [==============================] - 0s 817us/step - loss: 0.1616 - accuracy: 0.9987
Epoch 6/50
225/225 [==============================] - 0s 723us/step - loss: 0.1186 - accuracy: 0.9986
Epoch 7/50
225/225 [==============================] - 0s 771us/step - loss: 0.0897 - accuracy: 0.9992
Epoch 8/50
225/225 [==============================] - 0s 772us/step - loss: 0.0699 - accuracy: 0.9999
Epoch 9/50
225/225 [==============================] - 0s 774us/step - loss: 0.0565 - accuracy: 1.0000
Epoch 10/50
225/225 [==============================] - 0s 713us/step - loss: 0.046

testデータでのAcc

In [21]:
pred = model.predict(x_test)
accuracy_score(np.argmax(pred, axis=1), np.argmax(y_test, axis=1))

0.99875